In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.sample(4)

,sentiment,content
21162,happiness,crystaljanee greaaat minee pleaseeeeee
15357,worry,anyone else problem accessing account info ist...
13243,neutral,wordsaftersex started
1062,hate,goblinza can t access site


In [6]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.sample(4)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26516\395604862.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
2592,0,hungry
29232,1,thx tip figure one out
19052,0,missing bffls missing friday night date hot do...
12301,0,wish going see patmaine johnmaine garrettmaine...


In [9]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/aftabalam1210/mini-mlops-project.mlflow')
dagshub.init(repo_owner='aftabalam1210', repo_name='mini-mlops-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "aftabalam1210/mini-mlops-project"

Repository aftabalam1210/mini-mlops-project initialized!

2025/07/19 13:27:32 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7fadf46dbdd54ea99410889a514e6498', creation_time=1752911851825, experiment_id='1', last_update_time=1752911851825, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
from mlflow.data import from_pandas
import os

# Set MLflow experiment
mlflow.set_experiment("Logistic Regression Baseline")

with mlflow.start_run(run_name="Baseline Logistic Regression") as run:
    
    # ✅ Save the source dataset locally
    input_file_path = "baseline_tweet_emotions.csv"
    df.to_csv(input_file_path, index=False)
    
    # ✅ Log CSV file as artifact (so you can see & download it in MLflow UI)
    mlflow.log_artifact(input_file_path, artifact_path="input_data")

    # ✅ Log in-memory dataset as an MLflow input (for lineage)
    mlflow.log_input(from_pandas(df, source=input_file_path, name="logged_preprocessed_data"))
    
    # Vectorization & Split
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['content'])
    y = df['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log params and metrics
    mlflow.log_param("vectorizer", "CountVectorizer")
    mlflow.log_param("algorithm", "LogisticRegression")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log model
    mlflow.sklearn.log_model(model, "model")


e:\MLOps Learning\mlops-mini-project\.venv\lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'baseline_tweet_emotions.csv'. Exception: 
  return _dataset_source_registry.resolve(
e:\MLOps Learning\mlops-mini-project\.venv\lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
e:\MLOps Learning\mlops-mini-project\.venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to

🏃 View run Baseline Logistic Regression at: https://dagshub.com/aftabalam1210/mini-mlops-project.mlflow/#/experiments/1/runs/26b0f0ee796b43c7aaf8de51b4522bbb
🧪 View experiment at: https://dagshub.com/aftabalam1210/mini-mlops-project.mlflow/#/experiments/1
